In [ ]:
import os
from snowflake.snowpark import Session
import snowflake.snowpark.functions as F


def table_exists(session, schema='', name=''):
    """Check if a table exists in the schema."""
    exists = session.sql(f"""
        SELECT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES 
        WHERE TABLE_SCHEMA = '{schema}' AND TABLE_NAME = '{name}') AS TABLE_EXISTS
    """).collect()[0]['TABLE_EXISTS']
    return exists


def create_weather_table(session):
    """Create NOAA weather data table if it doesn't exist."""
    session.sql("""
        CREATE OR REPLACE TABLE WEATHER_DATA (
            DATE TIMESTAMP,
            ZIP_CODE STRING,
            STATION STRING,
            TEMPERATURE FLOAT,
            META_UPDATED_AT TIMESTAMP
        );
    """).collect()


def create_weather_stream(session):
    """Create a Snowflake stream for incremental changes."""
    session.sql("""
        CREATE OR REPLACE STREAM WEATHER_STREAM ON TABLE WEATHER_DATA;
    """).collect()


def merge_weather_updates(session):
    """Merge new weather data into the main table."""
    session.sql("ALTER WAREHOUSE HOL_WH SET WAREHOUSE_SIZE = XLARGE WAIT_FOR_COMPLETION = TRUE").collect()

    source = session.table("WEATHER_STREAM")
    target = session.table("WEATHER_DATA")

    updates = {
        "DATE": source["DATE"],
        "ZIP_CODE": source["ZIP_CODE"],
        "STATION": source["STATION"],
        "TEMPERATURE": source["TEMPERATURE"],
        "META_UPDATED_AT": F.current_timestamp()
    }

    target.merge(source, target["DATE"] == source["DATE"], [
        F.when_matched().update(updates),
        F.when_not_matched().insert(updates)
    ])

    session.sql("ALTER WAREHOUSE HOL_WH SET WAREHOUSE_SIZE = XSMALL").collect()


def main(session: Session) -> str:
    """Main function to create tables, process weather data, and merge updates."""
    if not table_exists(session, schema="PUBLIC", name="WEATHER_DATA"):
        create_weather_table(session)
        create_weather_stream(session)

    merge_weather_updates(session)

    return "Successfully processed NOAA Weather Data"


# For local debugging
# Be aware you may need to type-convert arguments if you add input parameters
if __name__ == '__main__':
    # Create a local Snowpark session
    with Session.builder.getOrCreate() as session:
        import sys
        if len(sys.argv) > 1:
            print(main(session, *sys.argv[1:]))  # type: ignore
        else:
            print(main(session))  # type: ignore

In [ ]:
if __name__ == '__main__':
    with Session.builder.getOrCreate() as session:
        import sys
        print(main(session))